In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from datetime import timedelta
import datetime as dtime
import json
import requests
import os
# rpy2 imports
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
pandas2ri.activate()
#plot imports
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns
import matplotlib.image as mpimg
from scipy import stats as sps
# R library
global eps
eps = importr("EpiEstim")

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')
from importdata import ImportData
from class_SEIRQ import SEIR
from datetime import datetime

/home/samuel/anaconda3/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


# Instantaneous Reproduction Number
https://www.rdocumentation.org/packages/EpiEstim/versions/2.2-3/topics/estimate_R


**Important, rpy2 2.9.4 version needed**

## Real Data

In [3]:
initdate = datetime(2020,5,15)
RM = ImportData(tstate ='13',initdate = initdate)
RM.importdata()

Importing General Data
Importing Population
Dlab Endpoint Error
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing ICU Beds Data from Minciencia
Importing Deaths by DEIS
Importing Active Infected by Minciencia
Done


In [4]:
df = pd.DataFrame({"dates":list(RM.I_d_r),"cases":RM.I_d_r_tr})
rdf = pandas2ri.py2ri(df)
r = robjects.r
window = 5  # always 5, test
start = r.seq(2, df.values.shape[0]-window)
end = r.seq( 2+ window,df.values.shape[0])

In [5]:
results = eps.estimate_R(rdf[1], method="parametric_si", config = eps.make_config(t_start = start,t_end = end,mean_si = 5,std_si = 2))

In [6]:
results2 = dict(results.items())
rhat = pandas2ri.ri2py(results2["R"])

In [7]:
rhat

,t_start,t_end,Mean(R),Std(R),Quantile.0.025(R),Quantile.0.05(R),Quantile.0.25(R),Median(R),Quantile.0.75(R),Quantile.0.95(R),Quantile.0.975(R)
0,2.0,7.0,7.809289,1.664946,4.894040,5.286796,6.631352,7.691290,8.858732,10.734380,11.394722
1,3.0,8.0,5.544378,1.047789,3.684199,3.940595,4.806382,5.478515,6.210636,7.372867,7.778680
2,4.0,9.0,4.163901,0.714103,2.883622,3.062931,3.662628,4.123150,4.620781,5.403896,5.675675
3,5.0,10.0,3.292244,0.520549,2.352027,2.485293,2.927813,3.264850,3.626829,4.192651,4.388091
4,6.0,11.0,2.728476,0.402292,1.997588,2.102160,2.447451,2.708730,2.987987,3.422155,3.571548
...,...,...,...,...,...,...,...,...,...,...,...
464,466.0,471.0,1.011098,0.019088,0.974030,0.979908,0.998159,1.010978,1.023906,1.042698,1.048849
465,467.0,472.0,1.011074,0.019067,0.974046,0.979918,0.998150,1.010955,1.023868,1.042639,1.048784
466,468.0,473.0,1.011051,0.019046,0.974062,0.979928,0.998140,1.010931,1.023831,1.042581,1.048719
467,469.0,474.0,1.011027,0.019025,0.974078,0.979938,0.998130,1.010907,1.023793,1.042523,1.048654


## Simulation Data

In [12]:
sim = SEIR(tsim=500,beta=0.15)

In [ ]:
sim.integr()

Import scikits-odes


In [11]:
df = pd.DataFrame({"dates":sim.t,"I":sim.I_d})
rdf = pandas2ri.py2ri(df)
r = robjects.r
window = 5  # always 5, test
start = r.seq(2, df.values.shape[0]-window)
end = r.seq( 2+ window,df.values.shape[0])

ValueError: If using all scalar values, you must pass an index

In [22]:
results = eps.estimate_R(rdf[1], method="parametric_si", config = eps.make_config(t_start = start,t_end = end,mean_si = 5,std_si = 2))

In [23]:
results2 = dict(results.items())
rhat = pandas2ri.ri2py(results2["R"])

In [24]:
rhat

,t_start,t_end,Mean(R),Std(R),Quantile.0.025(R),Quantile.0.05(R),Quantile.0.25(R),Median(R),Quantile.0.75(R),Quantile.0.95(R),Quantile.0.975(R)
0,2.0,7.0,7.299309,2.810950,2.873131,3.366715,5.260645,6.941822,8.949685,12.452232,13.753011
1,3.0,8.0,5.430897,1.879282,2.395690,2.750688,4.074141,5.215737,6.553785,8.845483,9.686901
2,4.0,9.0,4.158659,1.318901,1.989203,2.251543,3.210068,4.020093,4.956563,6.538679,7.114533
3,5.0,10.0,3.312501,0.976139,1.684313,1.886158,2.612663,3.217126,3.908587,5.064322,5.482081
4,6.0,11.0,2.750698,0.760804,1.467580,1.629762,2.206720,2.680883,3.218708,4.109875,4.430177
...,...,...,...,...,...,...,...,...,...,...,...
49990,49992.0,49997.0,1.014629,0.142438,0.754849,0.792246,0.915274,1.007972,1.106731,1.259725,1.312234
49991,49993.0,49998.0,1.014634,0.142458,0.754820,0.792221,0.915264,1.007975,1.106747,1.259764,1.312282
49992,49994.0,49999.0,1.014638,0.142477,0.754791,0.792196,0.915254,1.007977,1.106764,1.259804,1.312330
49993,49995.0,50000.0,1.014643,0.142497,0.754762,0.792171,0.915245,1.007980,1.106781,1.259844,1.312378


## Import endpoint data

In [25]:
endpoint = 'http://192.168.2.223:5006/getEffectiveReproductionAllComunas'

In [27]:
request = requests.get(endpoint)

In [35]:
R_eff = pd.DataFrame(request.json()['data'])
R_eff_dates = pd.DataFrame(request.json()['dates'])

In [37]:
R_eff_dates

,0
0,2020-04-07T04:00:00.000Z
1,2020-04-08T04:00:00.000Z
2,2020-04-09T04:00:00.000Z
3,2020-04-10T04:00:00.000Z
4,2020-04-11T04:00:00.000Z
...,...
209,2020-11-02T03:00:00.000Z
210,2020-11-03T03:00:00.000Z
211,2020-11-04T03:00:00.000Z
212,2020-11-05T03:00:00.000Z


In [38]:
aux = request.json()

In [42]:
R_eff_dates = [datetime.strptime(aux['dates'][i][:10],'%Y-%m-%d') for i in range(len(aux['dates']))]

In [43]:
R_eff_dates

[datetime.datetime(2020, 4, 7, 0, 0),
 datetime.datetime(2020, 4, 8, 0, 0),
 datetime.datetime(2020, 4, 9, 0, 0),
 datetime.datetime(2020, 4, 10, 0, 0),
 datetime.datetime(2020, 4, 11, 0, 0),
 datetime.datetime(2020, 4, 12, 0, 0),
 datetime.datetime(2020, 4, 13, 0, 0),
 datetime.datetime(2020, 4, 14, 0, 0),
 datetime.datetime(2020, 4, 15, 0, 0),
 datetime.datetime(2020, 4, 16, 0, 0),
 datetime.datetime(2020, 4, 17, 0, 0),
 datetime.datetime(2020, 4, 18, 0, 0),
 datetime.datetime(2020, 4, 19, 0, 0),
 datetime.datetime(2020, 4, 20, 0, 0),
 datetime.datetime(2020, 4, 21, 0, 0),
 datetime.datetime(2020, 4, 22, 0, 0),
 datetime.datetime(2020, 4, 23, 0, 0),
 datetime.datetime(2020, 4, 24, 0, 0),
 datetime.datetime(2020, 4, 25, 0, 0),
 datetime.datetime(2020, 4, 26, 0, 0),
 datetime.datetime(2020, 4, 27, 0, 0),
 datetime.datetime(2020, 4, 28, 0, 0),
 datetime.datetime(2020, 4, 29, 0, 0),
 datetime.datetime(2020, 4, 30, 0, 0),
 datetime.datetime(2020, 5, 1, 0, 0),
 datetime.datetime(2020, 5, 2

In [46]:
data = R_eff

In [45]:
tstate = '13'

In [47]:
            if len(tstate) == 2:
                R_eff = np.array(data.filter(regex='^'+tstate,axis=1).sum(axis=1))
            elif len(tstate) > 2:
                R_eff = np.array(data[tstate])

In [48]:
R_eff

array([list([3.5368909506998163, 2.627804793936648, 2.070233986954593, 1.7188233451627748, 1.473936229216061, 1.2995565999288903, 1.1568748515004013, 1.0671986908547952, 1.0165103857196347, 0.987658123986941, 0.9816676387536566, 0.9837695969368822, 0.9977773004024986, 1.0036046371416674, 1.0161954307509653, 1.0519949420071157, 1.1110697712022048, 1.1825322959964095, 1.2769321704050587, 1.4013475934742314, 1.543206507678476, 1.6861928348989283, 1.8188306281182423, 1.9309590264840981, 2.000729962458714, 1.9999956778977732, 1.9351832710554067, 1.8233693031576086, 1.6817667243089804, 1.5270559793486616, 1.3706523007137548, 1.229908927071825, 1.1233702946539323, 1.0644324813021568, 1.0670685075732025, 1.136805085736645, 1.2523013093334616, 1.378296521145573, 1.4581911786157598, 1.4682073409577252, 1.4179664769450555, 1.3449311645830584, 1.286445117598426, 1.2638041902161787, 1.2783757353462133, 1.3120099744325857, 1.337485960238086, 1.336447800823975, 1.304672096432646, 1.2505637247107135, 

In [49]:
data

,10101,10102,10103,10104,10105,10106,10107,10108,10109,10201,...,09202,09203,09204,09205,09206,09207,09208,09209,09210,09211
mean,"[3.733119343132012, 2.7356307610381467, 2.0257...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 10, 14.680783282228827, 14.538000007...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 10, 14.680783282228827, 14.538000007...","[6.113296132332245, 4.276523520586308, 2.87710...",...,"[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 14.680783282228827, 14.538000007368009, 9...","[5, 5, 5, 5, 10, 14.680783282228827, 14.538000...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.5504774640766652, 0.9703142583971188, 0.633..."
quantile0025,"[1.9289572397428154, 1.4566081662180481, 1.078...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[2.2434719372783687, 1.7193842550419662, 1.156...",...,"[0.1265890399214494, 0.1265890399214494, 0.126...","[1.2110463927198243, 3.0275304529955873, 3.961...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.1265890399214494, 0.1265890399214494, 0.126...","[0.5034358122227577, 0.2643779779407315, 0.130..."
quantile0975,"[6.1229498905230635, 4.411012065993089, 3.2663...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[11.88866156188192, 7.978449689112435, 5.36763...",...,"[18.444397270569677, 18.444397270569677, 18.44...","[27.858216954694488, 35.354691310548894, 31.86...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[18.444397270569677, 18.444397270569677, 18.44...","[3.1758704875112436, 2.1267525167080135, 1.526..."


In [50]:
            if len(tstate) == 2:
                R_eff = data.filter(regex='^'+tstate,axis=1).sum(axis=1)
            elif len(tstate) > 2:
                R_eff = (data[tstate])

In [51]:
R_eff

mean            [3.5368909506998163, 2.627804793936648, 2.0702...
quantile0025    [2.880855768157758, 2.1597359539819663, 1.7121...
quantile0975    [4.259214914129361, 3.1410972864986832, 2.4618...
dtype: object

In [53]:
endpointcounty = 'http://192.168.2.223:5006/getEffectiveReproductionAllComunas'
endpointstates = 'http://192.168.2.223:5006/getEffectiveReproductionAllStates'

In [56]:
auxcounty = requests.get(endpointcounty).json()
auxstate = requests.get(endpointstates).json()

In [59]:
datesstate = pd.DataFrame(auxstate['dates'])
datescounty = pd.DataFrame(auxcounty['dates'])

In [60]:
datesstate

,0
0,2020-04-05T04:00:00.000Z
1,2020-04-06T04:00:00.000Z
2,2020-04-07T04:00:00.000Z
3,2020-04-08T04:00:00.000Z
4,2020-04-09T04:00:00.000Z
...,...
211,2020-11-02T03:00:00.000Z
212,2020-11-03T03:00:00.000Z
213,2020-11-04T03:00:00.000Z
214,2020-11-05T03:00:00.000Z


In [61]:
datescounty

,0
0,2020-04-07T04:00:00.000Z
1,2020-04-08T04:00:00.000Z
2,2020-04-09T04:00:00.000Z
3,2020-04-10T04:00:00.000Z
4,2020-04-11T04:00:00.000Z
...,...
209,2020-11-02T03:00:00.000Z
210,2020-11-03T03:00:00.000Z
211,2020-11-04T03:00:00.000Z
212,2020-11-05T03:00:00.000Z


In [62]:
        datacounty = pd.DataFrame(auxcounty['data'])
        datastate = pd.DataFrame(auxstate['data'])

In [65]:
datastate['13']

mean            [2.7021522499025052, 2.004760155332217, 1.6658...
quantile0025    [2.5247528432498756, 1.8751974460078507, 1.561...
quantile0975    [2.885489331462001, 2.1385895186271964, 1.7736...
Name: 13, dtype: object

In [67]:
dataRM = datacounty.filter(regex='^13',axis=1)

In [88]:
suma = np.array([np.array(dataRM.loc['mean'][i]) for i in range(len(dataRM.loc['mean']))])

In [95]:
suma.sum(axis=0)/len(dataRM.loc['mean'])

array([4.50107517, 3.81173569, 3.25258259, 2.74918609, 1.9235974 ,
       1.58898957, 1.37107925, 1.195911  , 1.09540272, 0.99938189,
       1.16400839, 1.23692513, 1.22300483, 1.1191848 , 1.0333351 ,
       1.01548166, 1.05797906, 1.24674194, 1.42066119, 1.53116112,
       1.73745476, 2.14365839, 2.3581675 , 2.32054959, 2.1053693 ,
       1.92508618, 1.78885937, 1.67386078, 1.69522936, 1.89892648,
       1.94449173, 1.79751992, 1.58556559, 1.46908476, 1.41995958,
       1.41321542, 1.43953755, 1.47015897, 1.46792992, 1.43241886,
       1.59823967, 1.67641419, 1.69245194, 1.67969101, 1.66728436,
       1.66154248, 1.61407092, 1.53947761, 1.44038799, 1.33047817,
       1.23546694, 1.36023103, 1.41518898, 1.4014467 , 1.27518489,
       1.2132078 , 1.1813173 , 1.14095426, 1.08671803, 1.06127414,
       1.06093038, 1.07233265, 1.1058564 , 1.15084692, 1.18384919,
       1.20556171, 1.19977997, 1.18649731, 1.16741166, 1.14776852,
       1.12720085, 1.10107195, 1.07173887, 1.04486301, 1.02232

In [93]:
np.array(dataRM.loc['mean'][0]).sum()

226.93271821959326

In [97]:
datastate['13']

mean            [2.7021522499025052, 2.004760155332217, 1.6658...
quantile0025    [2.5247528432498756, 1.8751974460078507, 1.561...
quantile0975    [2.885489331462001, 2.1385895186271964, 1.7736...
Name: 13, dtype: object